In [ ]:
import pandas as pd 
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import pymongo

In [ ]:
#Browser setup
lp_url = "https://www.lonelyplanet.com/best-in-travel/countries"
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome")
browser.visit(lp_url)

In [ ]:
lp_soup = bs(browser.html, 'lxml')

In [ ]:
results = lp_soup.find_all("h1")

In [ ]:
top_countries = [result.get_text(strip=True).split('.') for result in results]
df = pd.DataFrame(top_countries)
df.columns=['rank', 'country']
df = df.iloc[1:,:]

In [ ]:
df['city'] = np.nan
df['source'] = "lonelyplanet"
df.columns= ['rank', 'country','city','source']

In [ ]:
lp_df = df
lp_df

In [ ]:
ins_url='https://www.thisisinsider.com/the-20-most-popular-travel-destinations-in-the-world-2016-9'

In [ ]:
response = requests.get(ins_url)

In [ ]:
ins_soup = bs(response.text, 'html.parser')
results = ins_soup.find_all('h2', class_="slide-title-text")

In [ ]:
data = []
for result in results:
    data.append(result.get_text(strip=True).replace('. ', ', ').split(","))

In [ ]:
ins_df = pd.DataFrame(data)

In [ ]:
ins_df.loc[8]=["12","Barcelona", "Spain"]
ins_df.loc[7]="13","Amsterdam", "Netherlands"
ins_df.loc[14]="6","Singapore", "Singapore"
ins_df.loc[15]="5","New York", "United States of America"

In [ ]:
ins_df.columns=['rank','city','country']
ins_df['source'] = 'insidertravel'

In [ ]:
ins_df[["rank"]] = ins_df[["rank"]].apply(pd.to_numeric)
ins_df= ins_df.sort_values(by="rank", ascending=True)
ins_df.reset_index(drop=True)

In [ ]:
# grabbed the states
url='https://alphabetizer.flap.tv/lists/list-of-states-in-alphabetical-order.php'
browser.visit(url)
html=browser.html
soup=bs(html,'lxml')

In [ ]:
us_states=[]
results=soup.find_all('li')
for x in range(len(results)):
    us_states.append(results[x].get_text())

In [ ]:
#cleaned up data
url='https://www.tripadvisor.com/TravelersChoice-Destinations-cTop-g1'

browser.visit(url)
html=browser.html
soup=bs(html,"html.parser")
rating=[]
city=[]
country=[]
id=[]
results=soup.find_all('div',class_='winnerLayer')
for result in results:
    rating.append(result.div.span.get_text())
    place=result.find('div',class_='winnerName').div.a.get_text()
    if len(place.split(',')) ==2:
        city.append(place.split(',')[0])
        country.append(place.split(',')[1])
    else:
        city.append(np.nan)
        country.append(place.split(',')[0])

for x in range(len(country)):
    if (country[x].strip() in us_states):
        country[x]='United States'
        if city[x]==np.nan:
            city[x]=country[x] 
    
ta_df=pd.DataFrame(dict(rank=rating,city=city,country=country,source="trip_advisor"))

In [ ]:
#cleaned up data
url="https://www.travelandleisure.com/trip-ideas/best-places-to-travel-in-2018"
browser.visit(url)
html=browser.html
soup=bs(html,'lxml')

In [ ]:
country=[]
city=[]
counter=0
results=soup.find_all('h3',class_='glide-slide-title')
check_for_doubles=""
for result in results:
    response=result.get_text()
    
    if check_for_doubles != response:
        if 'The 50 Best Places' not in response:

            response_split=response.strip().split(",")

            if len(response_split)==2:

                # two places added together with 'and' in country
                first_part=response_split[0].strip()
                second_part=response_split[1].strip()

                if ' and' in second_part:
                    
                    second_part=second_part.split(' and')[0]
                    
                country.append(second_part)   
                city.append(first_part)
            else:
                country.append(response_split[0].strip())
                city.append(np.nan)
                
        
        check_for_doubles=response
for x in range(len(country)):
    if (country[x] in us_states):
        country[x]='United States'
        if city[x]==np.nan:
            city[x]=country[x]
       
        
rating=[x for x in range(1,len(country)+1)]
tl_df=pd.DataFrame(dict(rank=rating,city=city,country=country,source="travel and leisure"))
tl_df

In [ ]:
frames = [lp_df,ins_df,tl_df, ta_df]
results = pd.concat(frames, sort=True)

In [ ]:
travel_data = results.reset_index(drop=True)

In [ ]:
travel_data

In [ ]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [ ]:
# Define database and collection
db = client.travel_db
collection = db.items

In [ ]:
def transform_data(row):
    data = row.to_dict()
    collection.insert_one(data)

In [ ]:
#df = df.reset_index()
travel_data.apply(transform_data, axis=1) 